In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import random
import xgboost as xgb

In [ ]:
datos = pd.read_csv('events_up_to_01062018.csv')
tabla_completa = datos.copy()
res = pd.read_csv('labels_training_set.csv');

In [ ]:
tabla_completa['aux']=1
#Voy a contar la cantidad para cada tipo de evento de cada persona
num_eventos = tabla_completa.groupby(['person','event'])['aux'].sum()

num_eventos = num_eventos.unstack().reset_index().fillna(0)
num_eventos = num_eventos.set_index('person')
num_eventos['total_eventos'] = num_eventos.sum(axis=1)
#normalizo los numeros obtenidos según la cantidad de eventos totales de esa persona
num_eventos_norm = num_eventos.loc[:,"ad campaign hit":"visited site"].div(num_eventos["total_eventos"], axis=0)
num_eventos_norm = num_eventos_norm.reset_index()
num_eventos_norm.head()

tabla_completa['timestamp'] = pd.to_datetime(tabla_completa['timestamp'])
#tabla_completa['timestamp'].max() -tabla_completa['timestamp'].min()

time_for_user = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max() - x.min())
time_for_user = time_for_user.reset_index()
time_for_user['total_horas'] = time_for_user['timestamp'].dt.total_seconds()/3600
time_for_user = time_for_user.drop(['timestamp'],axis='columns')

max_time = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max())
max_time = max_time.reset_index()
max_time['max'] = max_time['timestamp'].dt.month
max_time = max_time.drop(['timestamp'],axis='columns')

time_for_user = pd.merge(max_time,time_for_user,on='person',how='inner')

min_time = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.min())
min_time = min_time.reset_index()
min_time['min'] = min_time['timestamp'].dt.month
min_time = min_time.drop(['timestamp'],axis='columns')

time_for_user = pd.merge(min_time,time_for_user,on='person',how='inner')

num_eventos_norm = num_eventos_norm.reset_index()
num_eventos = num_eventos.reset_index()
total_eventos = pd.merge(num_eventos,num_eventos_norm, on='person',how='inner')
total_eventos = pd.merge(total_eventos,time_for_user,on='person',how='inner')

In [ ]:
total_eventos.shape

In [ ]:
tabla_encode = datos.copy()
tabla_encode.head()

In [ ]:
tabla_encode = tabla_encode.fillna(0)

In [ ]:
#device_type
tabla_encode.groupby('device_type')['person'].count()
#0 no cuenta (es que no hay evento)
#son solo 4 tipos distintos, así que los mantengo todos

In [ ]:
dev_type = tabla_encode[['person','device_type']]
dev_type['aux'] = 1
dev_type = dev_type.groupby(['person','device_type'])['aux'].count().unstack().reset_index().fillna(0)
dev_type = dev_type.drop([0],axis=1)
dev_type = dev_type.set_index('person')
dev_type = dev_type.add_prefix('device_type_')
dev_type = dev_type.reset_index()
dev_type.shape

In [ ]:
#device_type
tabla_encode.groupby('browser_version')['person'].count().sort_values(ascending=False)
#0 no cuenta (es que no hay evento)
#son 367 tipos distintos. Voy a vovler a mirar, pero uniendo por versiones

In [ ]:
test_b = tabla_encode['browser_version'].str.replace('\d+', '')
test_b = test_b.str.replace('.', '')
test_b = test_b.str.strip()
test_b2 = pd.DataFrame(test_b)
test_b2['aux']=1
test_b2 = test_b2.groupby('browser_version').count().reset_index().sort_values(by='aux',ascending=False)
#Ahora me quedan 30 browsers. Voy a quedarme con los 5 más usados. El resto irá a una columna "otros".
browsers = test_b2['browser_version'].head()
browsers

In [ ]:
browser_version = tabla_encode[['person','browser_version']]
browser_version['aux'] = 1
browser_version = browser_version.groupby(['person','browser_version'])['aux'].count().unstack().reset_index().fillna(0)
browser_version = browser_version.drop([0],axis=1)
browser_version = browser_version.set_index('person')
browser_version = browser_version.add_prefix('browser_version_')
browser_version = browser_version.reset_index()
browser_version.shape


In [ ]:
#screen_resolution
test_s = tabla_encode['screen_resolution']
test_s = pd.DataFrame(test_s)
test_s['aux']=1
test_s = test_s.groupby('screen_resolution').count().reset_index().sort_values(by='aux',ascending=False)
test_s = test_s.drop([0],axis=0)

resolutions = test_s['screen_resolution'].head()
resolutions
#Me quedo con las 5 resoluciones que mas buscan

In [ ]:
screen_resolution = tabla_encode[['person','screen_resolution']]
screen_resolution['aux'] = 1
screen_resolution = screen_resolution.groupby(['person','screen_resolution'])['aux'].count().unstack().reset_index().fillna(0)
screen_resolution = screen_resolution.drop([0],axis=1)
screen_resolution = screen_resolution.set_index('person')
screen_resolution = screen_resolution.add_prefix('screen_resolution_')
screen_resolution = screen_resolution.reset_index()
screen_resolution.shape


In [ ]:
#operating_system_version
test_os = tabla_encode['operating_system_version'].str.replace('\d+', '')
test_os = test_os.str.replace('.', '')
test_os = test_os.str.strip()
test_os2 = pd.DataFrame(test_os)
test_os2['aux']=1
test_os2 = test_os2.groupby('operating_system_version').count().reset_index().sort_values(by='aux',ascending=False)
test_os2

OSs = test_os2['operating_system_version'].head()
OSs
#Me quedo con las 5 resoluciones que mas buscan

In [ ]:
operating_system_version = tabla_encode[['person','operating_system_version']]
operating_system_version['aux'] = 1
operating_system_version = operating_system_version.groupby(['person','operating_system_version'])['aux'].count().unstack().reset_index().fillna(0)
operating_system_version = operating_system_version.drop([0],axis=1)
operating_system_version = operating_system_version.set_index('person')
operating_system_version = operating_system_version.add_prefix('operating_system_version_')
operating_system_version = operating_system_version.reset_index()
operating_system_version.shape


In [ ]:
#storage
test_st = tabla_encode['storage']
test_st = pd.DataFrame(test_st)
test_st['aux']=1
test_st = test_st.groupby('storage').count().reset_index().sort_values(by='aux',ascending=False)
test_st = test_st.drop([0],axis=0)

storages = test_st['storage'].head()
storages
#Me quedo con las 5 resoluciones que mas buscan

In [ ]:
storage = tabla_encode[['person','storage']]
storage['aux'] = 1
storage = storage.groupby(['person','storage'])['aux'].count().unstack().reset_index().fillna(0)
storage = storage.drop([0],axis=1)
storage = storage.set_index('person')
storage = storage.add_prefix('storage_')
storage = storage.reset_index()
storage.shape


In [ ]:
#condition
test_c = tabla_encode['condition']
test_c = pd.DataFrame(test_c)
test_c['aux']=1
test_c = test_c.groupby('condition').count().reset_index().sort_values(by='aux',ascending=False)
test_c = test_c.drop([0],axis=0)
test_c

#Ya son 5, me quedo todas.

In [ ]:
condition = tabla_encode[['person','condition']]
condition['aux'] = 1
condition = condition.groupby(['person','condition'])['aux'].count().unstack().reset_index().fillna(0)
condition = condition.drop([0],axis=1)
condition = condition.set_index('person')
condition = condition.add_prefix('condition_')
condition = condition.reset_index()
condition.shape

In [ ]:
#new_vs_returning
test_nv = tabla_encode['new_vs_returning']
test_nv = pd.DataFrame(test_nv)
test_nv['aux']=1
test_nv = test_nv.groupby('new_vs_returning').count().reset_index().sort_values(by='aux',ascending=False)
test_nv = test_nv.drop([0],axis=0)
test_nv
#Son solo 2 valores

In [ ]:
#Esta la hago un count.
new_vs_returning = tabla_encode[['person','new_vs_returning']]
new_vs_returning['aux'] = 1
new_vs_returning = new_vs_returning.groupby(['person','new_vs_returning'])['aux'].count().unstack().reset_index().fillna(0)
new_vs_returning = new_vs_returning.drop([0],axis=1)
new_vs_returning = new_vs_returning.set_index('person')
new_vs_returning = new_vs_returning.add_prefix('new_vs_returning_')
new_vs_returning = new_vs_returning.reset_index()
new_vs_returning.shape

In [ ]:
#search_engine
test_se = tabla_encode['search_engine']
test_se = pd.DataFrame(test_se)
test_se['aux']=1
test_se = test_se.groupby('search_engine').count().reset_index().sort_values(by='aux',ascending=False)
test_se = test_se.drop([0],axis=0)
test_se
#Son solo 4 valores

In [ ]:
#Esta la hago un count.
search_engine = tabla_encode[['person','search_engine']]
search_engine['aux'] = 1
search_engine = search_engine.groupby(['person','search_engine'])['aux'].count().unstack().reset_index().fillna(0)
search_engine = search_engine.drop([0],axis=1)
search_engine = search_engine.set_index('person')
search_engine = search_engine.add_prefix('search_engine_')
search_engine = search_engine.reset_index()
search_engine.shape

In [ ]:
#channel
test_ch = tabla_encode['channel']
test_ch = pd.DataFrame(test_ch)
test_ch['aux']=1
test_ch = test_ch.groupby('channel').count().reset_index().sort_values(by='aux',ascending=False)
test_ch = test_ch.drop([0],axis=0)
test_ch
#Mantengo los 7 valores

In [ ]:
#Esta la hago un count.
channel = tabla_encode[['person','channel']]
channel['aux'] = 1
channel = channel.groupby(['person','channel'])['aux'].count().unstack().reset_index().fillna(0)
channel = channel.drop([0],axis=1)
channel = channel.set_index('person')
channel = channel.add_prefix('channel_')
channel = channel.reset_index()
channel.shape

In [ ]:
#Ahora a unir todo...

In [ ]:
tabla_final = pd.merge(total_eventos,dev_type,on='person',how='inner')
tabla_final = pd.merge(tabla_final,screen_resolution,on='person',how='inner')
tabla_final = pd.merge(tabla_final,storage,on='person',how='inner')
tabla_final = pd.merge(tabla_final,condition,on='person',how='inner')
tabla_final = pd.merge(tabla_final,new_vs_returning,on='person',how='inner')
tabla_final = pd.merge(tabla_final,search_engine,on='person',how='inner')
tabla_final = pd.merge(tabla_final,channel,on='person',how='inner')
tabla_final = pd.merge(tabla_final,browser_version,on='person',how='left')
tabla_final = pd.merge(tabla_final,operating_system_version,on='person',how='left')
tabla_final = tabla_final.fillna(0)
tabla_final.shape

In [ ]:
tabla_final.info()

In [ ]:

intento_f = pd.merge(tabla_final,res,on='person',how='inner')

intento_f = intento_f.set_index('person')

In [ ]:
final = pd.read_csv('trocafone_kaggle_test.csv')
final = pd.merge(tabla_final,final,on='person',how='inner')
final = final.set_index('person')
x_entrega = final

In [ ]:
#XGBoost

X, y = intento_f.iloc[:,:-1],intento_f.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    
clf = LogisticRegression(random_state=124, solver='lbfgs', multi_class='multinomial')

model = clf.fit(X_train,y_train)


In [ ]:
#Veo la importancia de cada variable
from tabulate import tabulate
headers = ["name", "score"]
values = sorted(zip(datos_train.columns, model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))